In [1]:
import sys; sys.path.append('..')
from brain.schema import MultiModalInput, TextPromptInput
from brain.sdk import BrainClient

In [2]:
from pydantic import BaseModel


class FooSchema(BaseModel):
    answer:str
    explanation:str

brain_client = BrainClient("http://127.0.0.1:8000")

@brain_client.reasoner(schema=FooSchema)
def foo(system_prompt="You are helpful ai agent", user_prompt="what is 2+2"):
    return MultiModalInput(text=TextPromptInput(system_prompt=system_prompt, user_prompt=user_prompt))

In [3]:
import time

function_id = foo.register()
print(f"Registered Reasoner ID: {function_id}")


start_time = time.time()

# Use the function
result = brain_client.use(function_id)(system_prompt="You are helpful ai agent", user_prompt="what is 2+2")
print(f"Result: {result}")

end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Registered Reasoner ID: 1049fa58-a281-42f5-91f8-48af2bcbf135
Result: answer='4' explanation='The sum of 2 and 2 is calculated by adding the two numbers together, which yields 4.'
Execution time: 0.8443329334259033 seconds


In [4]:
start_time = time.time()
num_executions=5
# Use the function
futs = [brain_client.use(function_id,run_async=True)(system_prompt="You are helpful ai agent", user_prompt="what is 2+2") for _ in range(num_executions)]
results=[fut.result() for fut in futs]

end_time = time.time()
print(f"Wall time for 5 async: {end_time - start_time} seconds")

Wall time for 5 async: 1.3464808464050293 seconds


In [5]:
@brain_client.reasoner(schema=FooSchema)
def foo(user_prompt):
    return user_prompt

function_id = foo.register()
print(f"Registered Reasoner ID: {function_id}")

# Use the function
result = brain_client.use(function_id)(user_prompt="what is 2+2")
print(f"Result: {result}")

Registered Reasoner ID: f7132eac-27eb-40f0-bae5-90a89d81aac6


KeyboardInterrupt: 

In [ ]:
@brain_client.reasoner(schema=FooSchema)
def foo(user_prompt):
    return user_prompt,"you are a math genious, add to the explanation that 'i am a king'"

function_id = foo.register()
print(f"Registered Reasoner ID: {function_id}")

# Use the function
result = brain_client.use(function_id)(user_prompt="what is 2+2")
print(f"Result: {result}")

In [ ]:
import sys; sys.path.append('..')
from pydantic import BaseModel

from brain.sdk import BrainClient


class Story(BaseModel):
    next_line:str


brain_client = BrainClient("http://127.0.0.1:8000")

project=brain_client.project("Test2")

@brain_client.reasoner(name="Story Extender",schema=Story,project=project,tags=["story"])
def extender(story):
    return story,"You are the best story continuer, continue the story by adding one more lines."

extender_id = extender.register()

In [ ]:
@brain_client.multi_agent(name="story writer",project=project,tags=["story","writer"])
def story_writer(start, n=3):
    story = start
    for i in range(n):
        story += brain_client.use(extender_id)(story=story).next_line
    return story


@brain_client.multi_agent(name="story writer 4",project=project,tags=["story","writer","longer session/system"])
def story_writer(start, n=3):
    story = start
    for i in range(n):
        story += brain_client.use(extender_id)(story=story).next_line
    return story

In [ ]:
result = story_writer(start="This is a story of ",n=4)

In [ ]:
for i in result.split("."):
    print(i+"\n")

In [ ]:
project=brain_client.project("Test2")
brain_client.list_runs(project=project)

project=brain_client.project("Test1")
brain_client.list_runs(project=project)

In [ ]:
project=brain_client.project("Test2")
brain_client.list_reasoners(project=project)
brain_client.list_multiagents(project=project)

In [ ]:
print(brain_client.get_call_graph('77b43ccf-8929-4c64-ac0e-0090610d209b')['lineage'][2]['result'])

In [ ]:
brain_client.get_call_graph('77b43ccf-8929-4c64-ac0e-0090610d209b')

In [ ]:
from typing import List

from pydantic import BaseModel


# Define schemas for the reasoners
class Story(BaseModel):
    story: str

class Character(BaseModel):
    name: str
    description: str

class Action(BaseModel):
    story: str
    next_action: str

# Define reasoners
@brain_client.reasoner(schema=Story)
def extender(story):
    return story, "Continue the story by adding one more line."

extender_id = extender.register()

@brain_client.reasoner(schema=Character)
def character_describer(name):
    system_prompt = "You are a creative writer."
    user_prompt = f"Describe the character named {name} in a detailed and imaginative manner."
    return user_prompt,system_prompt

character_id = character_describer.register()

@brain_client.reasoner(schema=Action)
def action_extender(story):
    return story, "Suggest the next action in the story, keeping it engaging."

action_id = action_extender.register()

# Multi-Agent Workflow
@brain_client.multi_agent
def story_builder(start: str, characters: List[str], n: int = 3):
    # Initialize the story
    story = start
    
    # Step 1: Describe characters
    character_descriptions = [
        brain_client.use(character_id)(name=name).description
        for name in characters
    ]
    
    # Append character descriptions to the story
    for description in character_descriptions:
        story += f" {description}"
    
    # Step 2: Extend the story with new lines
    for _ in range(n):
        story += brain_client.use(extender_id)(story=story).story
    
    # Step 3: Add actions to the story
    for _ in range(n):
        story += brain_client.use(action_id)(story=story).next_action
    
    return story

# Example usage
result = story_builder(
    start="Once upon a time in a mystical forest,",
    characters=["Elara", "Wise Owl"],
    n=2
)
print(result)

In [ ]:
brain_client.list_runs()

In [ ]:
from brain.visualize import plot_workflow_matplotlib

lineage=brain_client.get_call_graph('ab36b24a-a474-4884-819c-3732ffd47173')
fig=plot_workflow_matplotlib(lineage['lineage'])

## Modifier

In [ ]:
import sys

sys.path.append('..')

In [ ]:
from pydantic import BaseModel

from brain.sdk import BrainClient



class FooSchema(BaseModel):
    answer:str
    explanation:str

brain_client = BrainClient("http://127.0.0.1:8000")

@brain_client.reasoner(schema=FooSchema)
def foo(system_prompt="You are helpful ai agent", user_prompt="what is 2+2"):
    return user_prompt,system_prompt

In [ ]:
import time
from brain.modifiers import COT

function_id = foo.register()

question = "How many R's are there in the word 'STRAWBERRY'?"

result = brain_client.use(function_id)(
    system_prompt="You are helpful ai agent", user_prompt=question
)
result_cot = brain_client.use(function_id, modifier=COT(num_steps=5))(
    system_prompt="You are helpful ai agent", user_prompt=question
)

print(f"Without COT : {result}\nWith COT : {result_cot}")

In [ ]:
start_time = time.time()
num_executions = 5
# Use the function
futs = [
    brain_client.use(function_id, modifier=COT(), run_async=True)(
        system_prompt="You are helpful ai agent", user_prompt=question
    )
    for _ in range(num_executions)
]
results = [fut.result() for fut in futs]

end_time = time.time()
print(f"Wall time for 5 async: {end_time - start_time} seconds")

In [ ]:
[result.answer for result in results]